In [1]:
import random
from string import ascii_lowercase


class BoardSize:
    
    def __init__(self):
        self._row = 0
        self._col = 0
        self._num_bombs = 0
        
    
    def set_size(self):
        
        self.print_colour = PrintInColour()
    
        level_int = self.get_player_select_level()
    
        if level_int == 4:
            while True:
                board_size = input("Set the board size in total amount of cells: ")
                num_bombs = input("Set the number of bombs: ")
                
                # conditions:
                # 1: both inputs must be int, board_size > 8, num_bombs > 0
                #   1_1: board_size - num_bombs > 7 
                
                if board_size.isdigit() and num_bombs.isdigit() and int(board_size) > 8 and int(num_bombs) > 0:
                    
                    if int(board_size) - int(num_bombs) > 7:
                        break
                        
                    else:
                        a_str = "The amount of cells should be more than bombs by 8 at least! Try again."
                        self.print_colour.red(a_str)
                        continue
                        
                else:
                    self.print_colour.invalid_input()
                    continue

            self._row, self._col = self.custom_setting(int(board_size))
            self._num_bombs = round(int(num_bombs))
            

        else:
            level_dict = {1: (9, 9, 10), 2: (16, 16, 40), 3: (30, 16, 99)}

            row, col, num_bombs = level_dict[level_int]
            
            self._row, self._col, self._num_bombs = row, col, num_bombs
            
        
    def get_player_select_level(self):
        
        welcome_message = ("\n"
                           "WELCOME TO BOMBSWEEPER GAME!\n"
                           "\n")
        
        game_level = ("\n"
                      "1. Beginner: 81 cells, 10 bombs\n"
                      "2. Intermediate: 256 cells, 40 bombs\n"
                      "3. Advance: 480 cells, 99 bombs\n"
                      "4. Custom\n"
                      "\n") 
        
        self.print_colour.lightblue(welcome_message)
        print(game_level)
        
        while True:
            level_str = input("Select game mode: ")
            level_str = level_str.strip(' ')
            if self.input_validation_level(level_str):
                return int(level_str)
            
            # the loop will keep going until player key in a valid number
            continue       

                   
    def input_validation_level(self, level_str):
        
        # check player's input of game mode selection 
        # valid input: 1 or 2 or 3 or 4 only

        if len(level_str) == 1 and level_str.isdigit():
            level_int = int(level_str)
            if level_int in range(1, 5):
                return True
        
        #self.print_colour.invalid_input()
        
        return False
    
    def custom_setting(self, board_size):
    
        num_r_c = board_size ** 0.5

        # max 26 alphabets(col)
        if num_r_c <= 26:
            return round(num_r_c), round(num_r_c)

        else:
            return round(board_size/26), 26
        
        

class BombArea():
    
    def __init__(self):
        Bs = BoardSize()
        Bs.set_size()

        self.row = Bs._row
        self.col = Bs._col
        self.num_bombs = Bs._num_bombs
        
        # underlying grid with bombs and value
        # a list of list, eg. 2*3 
        # [[0, 0, 0], [0, 0, 0]]
        # self.grid[0][0] is one cell (locaton to plant a bomb or value)
        self._grid = [[0 for i in range(self.col)] for i in range(self.row)]
    
    
    def set_grid(self):
        self.set_bombs()
        self.set_value()
        
        
    def set_bombs(self):
        
        set_bombs = 0
        
        while set_bombs < self.num_bombs:
            
            # -1 because last index of a list is always smaller than its length by 1
            bomb_row = random.randint(0, self.row-1)
            bomb_col = random.randint(0, self.col-1)
            
            # avoid planting on the same location again
            # if it's a bomb, the value will be -1 instead of 0
            if self._grid[bomb_row][bomb_col] != -1: 
                self._grid[bomb_row][bomb_col] = -1
                set_bombs += 1
                
    
    def get_neighbors(self, r, c):
        
        # check neighbouring cells
        # identify any bombs

        #| r-1, c-1 | r-1, c | r-1, c+1 |
        #| r,   c-1 | r,   c | r,   c+1 |
        #| r+1, c-1 | r+1, c | r+1, c+1 |
        
        neighbors = 0   
        
        # set max and min to make sure cells around are inside the grid
        for i in range(max(0, r-1), min(self.row, r+2)):
            for j in range(max(0, c-1), min(self.col, c+2)):
                
                if i == 0 and j == 0:
                    continue
                    
                elif self._grid[i][j] == -1:
                    neighbors += 1
            
        return neighbors
    
    
    def set_value(self):
        # traverse cells from the list of lists
        for r_num in range(self.row):
            for c_num in range(self.col):
                
                # cell that is not a bomb
                if self._grid[r_num][c_num] != -1: 
                    value = self.get_neighbors(r_num, c_num)
                    self._grid[r_num][c_num] = value

                    

class DisplayBoard(BombArea):
         
    def __init__(self):
        super().__init__()
        
        self.grid = self._grid
        
        self.layout = []
        self.print_colour = PrintInColour()
        
        
    def create_layout(self):
        # create an empty board
        for r in range (0, self.row):
            # each cell is a list type
            self.layout.append ([])

            for c in range (0, self.col):      
                self.layout[r].append ('-'.rjust(2) )
                # output is a list of lists: eg. 2*3: [['-', '-', '-'],['-', '-', '-']]


    def print_board(self):
        
        # pass to layout first before execute print_board()
        # self.layout[int(row)][int(col)] = 'item'

        x_str = '   '

        # display x-coordinates, the column index 
        for c in range (0, self.col):
            x_str += ' ' + (ascii_lowercase[c].rjust(3))
            
        x_str = '\n' + x_str
        self.print_colour.orange(x_str)
        
       
        # y-coordinates, the row index 
        for r in range (0, self.row):

            y = ' ' + str(r+1).rjust(2) 
            self.print_colour.orange(y)
            
            for c in range (0, self.col):
                y_str = '   '
                for c in range(0, self.col):
                    y_str += '  ' + self.layout[r][c]# eg. 0 - - - - -
            
            self.print_colour.green(y_str)
        print("\n")

        
        
    def open_cell(self, r, c):
    
        # a few scenarios:
        # 1: open the opened cell
        # 2: hit a bomb >> game over
        # 3: open a safe cell which has a value >= 0:
        
        # 1
        if self.layout[r][c] != ' -':
            a_str = "This cell is already opened!"
            self.print_colour.red(a_str)
            
        # 2   
        elif self.grid[r][c] == -1:
            
            a_str = ("Oops! You hit a bomb!\n"
                     "\n"
                     "GAME OVER...\n"
                     "\n")            
            
            self.layout[r][c] = ' *'
            self.print_board()
            self.reveal_board()
            self.print_colour.red(a_str)
            
            return True
              
        # 3   
        else:
            self.safe_cell(r, c)
            
            
    def safe_cell(self, r, c):
        
        # 1: layout[r][c] = the value
        # 2: if cell == 0:
        #  2_1: recursively open neighboring cells until the value is > 0
        #       everytime when run the loop, layout will be updated with new r, c
        
        val = self.grid[r][c] 
        
        self.layout[r][c] = str(val).rjust(2)
    
        if val == 0:
            
            for i in range(max(0, r-1), min(self.row, r+2)):
                for j in range(max(0, c-1), min(self.col, c+2)):
                    
                    # skip itself, which is the center cell
                    if i == r and j == c: 
                        continue
                        
                    # skip those cells that already opened
                    elif self.layout[i][j] != ' -':
                        continue

                    else:
                        self.safe_cell(i, j)
        
        
    def reveal_board(self):

        # assign all bombs from self.grid to self.layout
        for i in range(self.row):
            for j in range(self.col):

                val = self.grid[i][j]
                
                # if flagged wrongly
                if self.layout[i][j] == ' F' and self.grid[i][j] != -1:
                    self.layout[i][j] = str(val).rjust(2)
                
                # if unopened cell and not flagged is a bomb
                elif self.layout[i][j] == ' -' and self.grid[i][j] == -1:                 
                    self.layout[i][j] = ' *'.rjust(2) 

                else:
                    continue
                    
            
    def set_flag(self, r, c):

    # 1: if set flag on an unopened cell ==> mark 'F'
    # 2: if flag on a flagged cell ==> remove 'F'

        if self.layout[r][c] == ' -' and self.layout[r][c] != ' F':
            self.layout[r][c] = 'F'.rjust(2)

        elif self.layout[r][c] == ' F':
            self.layout[r][c] = '-'.rjust(2)

        else:
            a_str = "This cell cannot be flagged!"
            self.print_colour.red(a_str)
        
        
    def hint(self):
        
        unopen_location = [] 
        # output: a list of tuples, eg.[(1, 2), (0, 7)]
        
         
        for i in range(self.row):
            for j in range(self.col):
                if self.layout[i][j] == ' -' and self.grid[i][j] >= 0:
                    
                    unopen_location.append((i,j))
        
        # get a random pair of coordinates from unopen_location
        if len(unopen_location) > 0:
            index = random.randint(0, len(unopen_location)-1)
            random_hint = unopen_location[index] 
            
            x = random_hint[0]
            y = random_hint[1]
            
            # convert number to alphabet
            y_alpha = chr(y + 97)
            a_str = "{}{} has opened!".format(x+1, y_alpha)
           
            self.print_colour.purple(a_str)
            
            self.open_cell(x, y)
            
        else:
            a_str = "Check those flagged locations! You may put wrongly"
            self.print_colour.purple(a_str)
           
        
    def bomb_left(self):
        
        counter = 0
        
        for i in range(self.row):
            for j in range(self.col):
                if self.layout[i][j] == ' F':
                    counter += 1
                    
        # total bombs - total flagged cells            
        bomb_left = self.num_bombs - counter
        a_str = "bombs remaining: {}".format(bomb_left)
            
        self.print_colour.lightblue(a_str)
        print("\n")



class BombSweeper:
    
    # class attributes
    instructions =( "\n"
                    "                            How To Play\n"
                    "=========================================================================\n"
                    "\n"
                    "           key in the row <space> column to open\n"
                    "           followed by <space> f to put or remove a flag \n"
                    "\n"
                    "         Example: 1 a  ==> open the 1st row and 1st collumn\n"
                    "         Example: 3 b f ==> put a flag on 3rd row and 2nd collumn\n"
                    "\n"
                    "          how to play: 'i', help: 'h', restart: 'r', quit: 'q' \n"
                    "\n"
                    "=========================================================================\n")

    symbols = "              " + "'*': bombs   'F': flag   '-': unopened   '0': empty"

    
    def __init__(self):
        
        # instance attributes
        self.board = DisplayBoard()
        self.row = self.board.row
        self.col = self.board.col
        self.num_bombs  = self.board.num_bombs
        self.layout = self.board.layout
        
        self.print_colour = PrintInColour()
        
        self.board.set_grid()
        self.board.create_layout()
        
        self.run_game()
        self.play_again()
        
        
    def get_player_rc(self):
        
        while True:
            
            player_input = input("Where would you like to open or put a flag? ")

            #         str ==> list of str
            # eg."1 22 F" ==> ['1', '22', 'F']
            lst = player_input.strip(' ')
            lst = lst.split(' ')

            if self.input_validation_rc(lst):
                return lst
            
            continue
            
    
    def input_validation_rc(self, lst):
        
        if len(lst) == 2 and lst[0].isdigit() and lst[1].isalpha() and len(lst[1]) == 1:

            # convert alphabet to number
            if 0 < int(lst[0]) < self.row+1 and ord(lst[1].lower())-97 < self.col:
                
                return True

        elif len(lst) == 3 and lst[0].isdigit() and lst[1].isalpha() and len(lst[1]) == 1:
            if 0 < int(lst[0]) < self.row+1 and ord(lst[1].lower())-97 < self.col:
                if lst[2].lower() == 'f':
                    
                    return True
                
        elif len(lst) == 1 and (lst[0].lower() == 'i' or lst[0].lower() == 'h' or lst[0].lower() == 'q' or lst[0].lower() == 'r'):
            return True
        
        # else
        PrintInColour().invalid_input()
        
        return False
    
    
    def take_player_input(self):
        
        player_input = self.get_player_rc()
        
        if len(player_input) == 1:
            
            p = player_input[0].lower()
            
            if p == 'i':
                self.print_colour.orange(self.instructions)
                
            elif p == 'h':
                self.board.hint()
                
            elif p == 'r':
                BombSweeper()
                
            elif p == 'q':
                while True:
                    a_str = "Are you sure you want to quit? y/n "
                    p = input("\033[31m {}\033[00m".format(a_str))
                    
                    if p.lower() == 'y':
                        return True
                
                    elif p.lower() == 'n':
                        return False

                    else:
                        self.print_colour.invalid_input()
                        continue
                        
        else:
            r = int(player_input[0])-1
            c = ord(player_input[1].lower())-97
                        
            if len(player_input) == 2:
                self.board.open_cell(r, c)

            else:
                self.board.set_flag(r, c)

            
    def win_or_lose(self):
        unopened = 0
        
        for i in range(self.row):
            for j in range(self.col):
                if self.layout[i][j] == ' -' or self.layout[i][j] == ' F':
                    unopened += 1
                    
                # lose    
                elif self.layout[i][j] == ' *':
                    return True
        # win
        if unopened == self.num_bombs:         
            a_str = "VICTORY!"
            self.print_colour.orange(a_str)
            return True
        
        else:
            return False
        
                    
    def play_again(self):
        
        while True:

            play_again = input("Would you like to start a new game? y/n ")

            if play_again.lower() == "y":
                BombSweeper()
                break

            elif play_again.lower() == "n":
                a_str = ("\n"
                         "GOOD BYE!\n"
                         "\n")
    
                self.print_colour.red(a_str)
                return False
                
            else:
                self.print_colour.invalid_input()
                continue
                
    
    def run_game(self):

        self.print_colour.orange(self.instructions)
        self.print_colour.lightblue(self.symbols)

        while True:
            
            self.board.print_board() 
            
            self.board.bomb_left()
            
            if self.take_player_input():
                break
            
            elif self.win_or_lose():
                self.board.print_board()
                break
                

                
# for display purpose
class PrintInColour:
    
    def red(self, text):
        print('\033[31m', text, '\033[0m', sep='')
    
    def green(self, text):
        print('\033[32m', text, '\033[0m', sep='')
    
    def orange(self, text):
        print('\033[33m', text, '\033[0m', sep='')
    
    def purple(self, text):
        print('\033[35m', text, '\033[0m', sep='')
    
    def lightblue(self, text):
        print('\033[36m', text, '\033[0m', sep='')
        
    def invalid_input(self):
        invalid = "Invalid input! Try again."
        self.red(invalid)

        

In [5]:
BombSweeper()


WELCOME TO BOMBSWEEPER GAME!



1. Beginner: 81 cells, 10 bombs
2. Intermediate: 256 cells, 40 bombs
3. Advance: 480 cells, 99 bombs
4. Custom


Select game mode: 1

                            How To Play

           key in the row <space> column to open
           followed by <space> f to put or remove a flag 

         Example: 1 a  ==> open the 1st row and 1st collumn
         Example: 3 b f ==> put a flag on 3rd row and 2nd collumn

          how to play: 'i', help: 'h', restart: 'r', quit: 'q' 


              '*': bombs   'F': flag   '-': unopened   '0': empty

      a   b   c   d   e   f   g   h   i
  1
      -   -   -   -   -   -   -   -   -
  2
      -   -   -   -   -   -   -   -   -
  3
      -   -   -   -   -   -   -   -   -
  4
      -   -   -   -   -   -   -   -   -
  5
      -   -   -   -   -   -   -   -   -
  6
      -   -   -   -   -   -   -   -   -
  7
      -   -   -   -   -   -   -   -   -
  8
      -   -   -   -   -   -   -   -   -
  9
      -   -   -   -   -   - 

Would you like to start a new game? y/n y

WELCOME TO BOMBSWEEPER GAME!



1. Beginner: 81 cells, 10 bombs
2. Intermediate: 256 cells, 40 bombs
3. Advance: 480 cells, 99 bombs
4. Custom


Select game mode: 1

                            How To Play

           key in the row <space> column to open
           followed by <space> f to put or remove a flag 

         Example: 1 a  ==> open the 1st row and 1st collumn
         Example: 3 b f ==> put a flag on 3rd row and 2nd collumn

          how to play: 'i', help: 'h', restart: 'r', quit: 'q' 


              '*': bombs   'F': flag   '-': unopened   '0': empty

      a   b   c   d   e   f   g   h   i
  1
      -   -   -   -   -   -   -   -   -
  2
      -   -   -   -   -   -   -   -   -
  3
      -   -   -   -   -   -   -   -   -
  4
      -   -   -   -   -   -   -   -   -
  5
      -   -   -   -   -   -   -   -   -
  6
      -   -   -   -   -   -   -   -   -
  7
      -   -   -   -   -   -   -   -   -
  8
      -   -   -   -   -   -   

In [ ]:
BombSweeper()

In [ ]:
BombSweeper()